<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-импорт-библиотек-и-функций" data-toc-modified-id="Загрузка-и-импорт-библиотек-и-функций-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и импорт библиотек и функций</a></span></li><li><span><a href="#Загрузка-и-изучение-данных-из-файла" data-toc-modified-id="Загрузка-и-изучение-данных-из-файла-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка и изучение данных из файла</a></span></li><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Предобработка текста</a></span></li><li><span><a href="#Создание-эмбеддингов" data-toc-modified-id="Создание-эмбеддингов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Создание эмбеддингов</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Построение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-&quot;Случайный-лес&quot;" data-toc-modified-id="Модель-&quot;Случайный-лес&quot;-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель "Случайный лес"</a></span></li><li><span><a href="#Модель-LightGBM" data-toc-modified-id="Модель-LightGBM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель LightGBM</a></span></li></ul></li><li><span><a href="#Тестирование-моделей" data-toc-modified-id="Тестирование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование моделей</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Классификация и поиск негативных комментариев (с BERT)

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

В проекте требуется создать модель для классификации комментариев на позитивные и негативные (токсичные). Для работы предоставлен набор данных с разметкой о токсичности правок. Построенная модель должна иметь значение метрики качества *F1* не меньше 0.75. 

## Подготовка данных

### Загрузка и импорт библиотек и функций

In [1]:
!pip install lightgbm -U

In [2]:
!pip install transformers -U

In [3]:
#Раскомментить только при необходимости! При установке библиотек будет подкачиваться 2-3 Гб информации. Процесс не быстрый
#!pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu117 -U

In [4]:
import pandas as pd
import numpy as np
import torch
import transformers as tfs
from tqdm import notebook
import time
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score, make_scorer
from sklearn.utils import shuffle

### Загрузка и изучение данных из файла

Загрузим данные из файла и ознакомимся с таблицей.

In [5]:
try:
    main_data = pd.read_csv('toxic_comments.csv', index_col=[0])
except:
    main_data = pd.read_csv('/datasets/toxic_comments.csv')
main_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Теперь посмотрим непосредственно на сами данные. Изменим отображаемую по умолчанию ширину столбцов в pandas, чтобы увидеть больше текстовой информации.

In [6]:
pd.options.display.max_colwidth = 800
main_data.head(10)

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,alignment on this subject and which are contrary to those of DuLithgow,0


Данные отображены, можно приступать к обработке текста.

### Предобработка текста

Оставим в тексте только слова на латинице. Остальные символы заменим пробелами. Оформим код в виде функции.

In [7]:
def clear_text(text):
    text_list = re.sub(r'[^a-zA-Z]', ' ', text).split()
    return " ".join(text_list)

In [8]:
main_data['text'] = main_data['text'].apply(clear_text)

Теперь следует привести весь текст к нижнему регистру. 

In [9]:
main_data['text'] = main_data['text'].str.lower()

In [10]:
main_data.head(7)

,text,toxic
0,explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now,0
1,d aww he matches this background colour i m seemingly stuck with thanks talk january utc,0
2,hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0
3,more i can t make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it s listed in the relevant form eg wikipedia good article nominations transport,0
4,you sir are my hero any chance you remember what page that s on,0
5,congratulations from me as well use the tools well talk,0
6,cocksucker before you piss around on my work,1


Преобразования текста прошли верно.

Выполним теперь поиск дубликатов в тексте и удалим их, если они есть.

In [11]:
main_data.duplicated().sum()

1294

In [12]:
main_data.drop_duplicates(inplace=True)
main_data.duplicated().sum()

0

Дубликаты успешно удалились, но проверим отдельно наличие дубликатов только в столбце с текстом без учёта других.

In [13]:
main_data['text'].duplicated().sum()

29

Данная ситуация говорит о том, что в таблице есть одинаковые тексты, но с разной оценкой токсичности. Такие тексты следует удалить, чтобы не запутывать модель. Отфильтруем такие дубликаты вместе с исходными строками, для чего применим параметр *keep=False*.

In [14]:
print(main_data['text'].duplicated(keep=False).sum())
main_data = main_data[~main_data['text'].duplicated(keep=False)]

58


In [15]:
main_data.shape

(157940, 2)

Фильтрация проша успешно.

### Создание эмбеддингов

Для перевода текста в векторное представление (эмбеддинги) воспользуемся одним из аналогов модели BERT - DistilBERT. Данная модель весит на 40% меньше, чем оригинальная BERT-модель, работает на 60% быстрее ее и сохраняет 97% ее функциональности. Веса модели и конфигурации используем из готовой предобученной на токсичность текста модели *dapang/distilbert-base-uncased-finetuned-toxicity*.

Модель построим с помощью средств библиотеки transformers и torch. До создания эмбеддингов произведём токенизацию текста и преобразование в формат тензоров. Все действия оформим в виде функции, которая будет в дальнейшем создавать преобразованные обучающие признаки (в формате Dataframe).

In [16]:
def make_embeddings(features): 
    #Создание устройства, для подключения видеокарты к расчётам
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
    # Создание токенизатора и токенизация текста
    tokenizer = tfs.DistilBertTokenizer.from_pretrained('dapang/distilbert-base-uncased-finetuned-toxicity')
    tokenized= features.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
    
    #Поиск длины наибольшего вектора
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)

    #Создание отступов и масок
    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)

    #Создание модели
    model = tfs.DistilBertModel.from_pretrained('dapang/distilbert-base-uncased-finetuned-toxicity')
    model = model.to(device)
    batch_size = 50

    #Создание эмбеддингов
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        batch_embedding = batch_embeddings[0][:,0,:]
        embeddings.append(batch_embedding.cpu().numpy())

    features_converted = np.concatenate(embeddings)
    features_converted = pd.DataFrame(features_converted)
    return features_converted

Для того чтобы эмбеддинги не создавались слишком долго, сделаем случайную выборку размером 20000 из исходных данных.

In [17]:
data_p = main_data.sample(20000, random_state=1234)

In [18]:
data_p.shape

(20000, 2)

Выборка создалась корректно. Можно приступать к созданию эмбеддингов.

In [19]:
%%time

features = make_embeddings(data_p['text'])

Some weights of the model checkpoint at dapang/distilbert-base-uncased-finetuned-toxicity were not used when initializing DistilBertModel: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/400 [00:00<?, ?it/s]

Wall time: 10min 46s


### Подготовка признаков

Теперь подготовим обучающие и тестовые признаки. Данные с обычными (не целевыми) признаками уже были подготовлены в предыдущем пункте. Создадим целевой признак. Перед разделением на выборки исследуем баланс классов.

In [20]:
data_p['toxic'].value_counts()

0    18042
1     1958
Name: toxic, dtype: int64

Имеет место дисбаланс классов, единичных значений почти в 9 раз меньше. Этот факт надо учесть при создании выборки. Воспользуемся параметром *stratify* в функции *train_test_split*.

In [21]:
target = data_p['toxic'].reset_index(drop=True)

#выделим тестовую выборку размером в 20%
train_valid_features, test_features, train_valid_target, test_target = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state=123)

#выделим валидационную выборку размером в 20% от исходной выборки (test_size = 0.2/0.8)
train_features, valid_features, train_target, valid_target = train_test_split(
    train_valid_features, train_valid_target, test_size=0.25, stratify=train_valid_target, random_state=123)

print('Размер обучающей выборки признаков:', train_features.shape)
print('Размер обучающей выборки целевых признаков:', train_target.shape)
print('Размер валидационной выборки признаков:', valid_features.shape)
print('Размер валидационной выборки целевых признаков:', valid_target.shape)
print('Размер тестовой выборки признаков:', test_features.shape)
print('Размер тестовой выборки целевых признаков:', test_target.shape)

Размер обучающей выборки признаков: (12000, 768)
Размер обучающей выборки целевых признаков: (12000,)
Размер валидационной выборки признаков: (4000, 768)
Размер валидационной выборки целевых признаков: (4000,)
Размер тестовой выборки признаков: (4000, 768)
Размер тестовой выборки целевых признаков: (4000,)


Так как имеет место дисбаланс значений целевого признака, то выполним увеличение обучающей выборки, добавив объекты с единичными значениями целевого признака.

In [22]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=123)
    return features_upsampled, target_upsampled

train_features_ups, train_target_ups = upsample(train_features, train_target, 8)
print(train_features_ups.shape)
print(train_target_ups.shape)
print(train_target_ups.value_counts())

(20225, 768)
(20225,)
0    10825
1     9400
Name: toxic, dtype: int64


Увеличение выборки выполнено корректно. Подготовка признаков на этом закончена, можно приступать к построению моделей.

## Построение моделей

### Модель логистической регрессии

Построим модель логистической регрессии и в цикле подберём один из её гиперпараметров. Качество модели проверим на валидационной выборке.

In [23]:
c_values = list(np.linspace(0.001, 100, 10))
params_grid = {'C': c_values,
             }

In [24]:
%%time

best_score = 0
best_params = {}
for c_value in params_grid['C']:
    params = {
        'solver': 'liblinear',
        'C': c_value,
        #'class_weight': 'balanced',
        'random_state': 123
    }
    model = LogisticRegression(**params)
    model.fit(train_features_ups, train_target_ups)
    predictions = model.predict(valid_features)
    result_score = f1_score(valid_target, predictions)
    if result_score > best_score:
        best_params = params
        best_score = result_score

Wall time: 3min


In [25]:
print('Среднее значение метрики F1: ', best_score)
print('Лучшие параметры модели:')
print(best_params)

Среднее значение метрики F1:  0.7362514029180697
Лучшие параметры модели:
{'solver': 'liblinear', 'C': 33.333999999999996, 'random_state': 123}


Сохраним все параметры модели в словаре, чтобы использовать в разделе 3. 

In [26]:
best_params_voc = {}
best_params_voc['logistic'] = best_params

**Вывод:** модель логистической регрессии показала неплохой результат. Но всё равно рассмотрим далее другие модели.

### Модель "Случайный лес"

Построим модель "Случайный лес" и в цикле подберём её гиперпараметры. Качество модели проверим на валидационной выборке.

In [27]:
params_grid = {'n_estimators': range(10,101,10),
               'max_depth': range(1,10,1)
             }

In [28]:
%%time

best_score = 0
best_params = {}
for n_estimators in params_grid['n_estimators']:
    for max_depth in params_grid['max_depth']:
        params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            #'class_weight': 'balanced',
            'random_state': 123,
            'n_jobs': -1
            }
        model = RandomForestClassifier(**params)
        model.fit(train_features_ups, train_target_ups)
        predictions = model.predict(valid_features)
        result_score = f1_score(valid_target, predictions)
        if result_score > best_score:
            best_params = params
            best_score = result_score

Wall time: 55.1 s


In [29]:
print('Значение метрики F1:', best_score)
print('Лучшие параметры модели:')
print(best_params)

Значение метрики F1: 0.7479674796747968
Лучшие параметры модели:
{'n_estimators': 10, 'max_depth': 9, 'random_state': 123, 'n_jobs': -1}


Сохраним все параметры модели в словаре, чтобы использовать в разделе 3. 

In [30]:
best_params_voc['forest'] = best_params

**Вывод:** модель "Случайный лес" показала ожидаемо ещё более высокий результат, чем модель логистической регрессии. Рассмотрим далее модель градиентного бустинга.

### Модель LightGBM

Построим модель LightGBM и в цикле подберём её гиперпараметы. Качество модели проверим на валидационной выборке.

In [31]:
params_grid = {'n_estimators': [500],
              'num_leaves': [27, 29, 30],
              'learning_rate':[0.06, 0.07, 0.09]
             }

In [32]:
%%time

best_score = 0
best_params = {}
for n_estimators in params_grid['n_estimators']:
    for num_leaves in params_grid['num_leaves']:
        for learning_rate in params_grid['learning_rate']:
            params = {
                'n_estimators': n_estimators,
                'num_leaves': num_leaves,
                'learning_rate': learning_rate,
                #'class_weight': 'balanced',
                'random_state': 123
                }
            model = LGBMClassifier(**params)
            model.fit(train_features_ups, train_target_ups)
            predictions = model.predict(valid_features)
            result_score = f1_score(valid_target, predictions)
            if result_score > best_score:
                best_params = params
                best_score = result_score

Wall time: 1min 7s


In [33]:
print('Значение метрики F1:', best_score)
print('Лучшие параметры модели:')
print(best_params)

Значение метрики F1: 0.772169167803547
Лучшие параметры модели:
{'n_estimators': 500, 'num_leaves': 27, 'learning_rate': 0.09, 'random_state': 123}


Сохраним все параметры модели в словаре, чтобы использовать в разделе 3. 

In [34]:
best_params_voc['lightgbm'] = best_params

**Вывод:** модель LightGBM лучший результ среди всех моделей. Требуемое значение метрики F1 достигнуто.

## Тестирование моделей

Гиперпараметры для рассмотренных моделей успешно подобраны.Также получены значения метрики F1 на валидационной выборке. Теперь проверим работу моделей на тестовой выборке и получим уже другие значения метрики F1, а также посчитаем время обучения и предсказания для каждой модели.

Для удобства создадим таблицу для хранения результатов.

In [35]:
model_results = pd.DataFrame({'Модель': [], 'F1-мера':[], 'Время обучения, с': [], 'Время предсказания, с':[]})

Cоздадим функцию, которая будет обучать модель, получать предсказания по **тестовой** выборке и рассчитывать время этих операций.

In [36]:
def model_routine(model, train_features, train_target, test_features, test_target):
    time1 = time.time()
    model.fit(train_features, train_target)
    time2 = time.time()
    fit_time = time2 - time1

    time1 = time.time()
    predictions = model.predict(test_features)
    time2 = time.time()
    predict_time = time2 - time1 
    
    f1_value = f1_score(test_target, predictions)
    
    return f1_value, fit_time, predict_time

Последовательно построим модели с подобранными гиперпараметрами, посчитаем необходимые величины и занесём их в таблицу.

In [37]:
model = LogisticRegression(**best_params_voc['logistic'])
f1_value, fit_time, predict_time = model_routine(model, train_features_ups, train_target_ups, test_features, test_target)
model_results.loc[0] = ['Logistic Regression', f1_value, fit_time, predict_time]

model = RandomForestClassifier(**best_params_voc['forest'])
f1_value, fit_time, predict_time = model_routine(model, train_features_ups, train_target_ups, test_features, test_target)
model_results.loc[1] = ['Random Forest Classifier', f1_value, fit_time, predict_time]

model = LGBMClassifier(**best_params_voc['lightgbm'])
f1_value, fit_time, predict_time = model_routine(model, train_features_ups, train_target_ups, test_features, test_target)
model_results.loc[2] = ['LightGBM', f1_value, fit_time, predict_time]

Посмотрим теперь таблицу с численными результатами.

In [38]:
model_results

,Модель,F1-мера,"Время обучения, с","Время предсказания, с"
0,Logistic Regression,0.713178,19.544148,0.015617
1,Random Forest Classifier,0.730455,0.250017,0.015626
2,LightGBM,0.762689,6.979536,0.031252


**Вывод:** наибольшее значение F1-меры на тестовой выборке показала модель LightGBM. При этом у данной модели не самое большое время обучения. По условиям задачи значение F1-меры должно быть не менее 0,75. Модель LightGBM превысила это значение, поэтому выбираем эту модель для поставленной в проекте задачи.

## Общий вывод

В проекте были загружены и изучены данные с текстовыми комментариями. 

Произведена предобработка текста: текст очищен от лишних символов кроме латинских букв, все символы приведены к нижнему регистру, удалены дубликаты.

Далее текст переведён в векторное представление (эмбеддинги) с помощью модели DistilBERT на основе предобученной модели для токсичных текстов. Ввиду ограниченности производительной мощности, для создания эмбеддингов была использована уменьшенная выборка из исходных данных. 

Следующим шагом подготовлены признаки для обучения моделей: выборка разделена на обучающую и тестовую, выделены обучающие и целевые признаки, проведено увеличение выборки из-за дисбаланса классов.

Затем было произведено построение трёх моделей: модель логистической регрессии, модель "Случайный лес" и модель градиентного бустинга LightGBM. Предсказания моделей выполнялись на обучающей выборке, а метрика F1 считалась на валидационной выборке. Наилучшие результаты по качеству предсказания показала модель LightGBM. 

Далее была проверена работа всех моделей на тестовой выборке. Наилучший результат по метрике F1 показала модель LightGBM. При этом результат превышает требуемое в проекте значение 0,75.

Для поставленной в проекте задачи выбрана модель LightGBM.